In [1]:
# Import main packages
import sys, os
sys.path.insert(1, os.path.abspath('..') )
import ECOdiagnostics as eco
from ECOdiagnostics.utils import config_parser
import xarray as xr
import xgcm
import numpy as np

# Import matplotlib for plotting and functions to override default values
import matplotlib.pyplot as plt
from ECOanalysis import update_plotstyle, cmap_OB
cmap_OB=cmap_OB(); update_plotstyle()

In [2]:
exp = 'EXP00_5d50'
path_to_dataset = 'data/'+exp+'/' # Here chosen as relative path, adjust if notebook is executed outside of 
file_name_proc = exp+'_proc*'       # Import all files with this prefix
file_name_pre = exp+'_pre*'       # Import all files with this prefix
ds     = xr.open_mfdataset(path_to_dataset+file_name_proc, decode_times=False, parallel=True, data_vars='minimal') 
ds_raw = xr.open_mfdataset(path_to_dataset+file_name_pre,  decode_times=False, parallel=True, data_vars='minimal') 

# get configuration from processing for matching parameters
kwargs_proc, kwargs_pre, kwargs_sim = config_parser(config_path='../ECOprocessing/Configs/'+exp, sub_config='sub_5d50')

# define xgcm configuration based on dataset metrics
_metrics = {('X',): ['e1tm', 'e1um', 'e1vm', 'e1fm'],
     ('Y',): ['e2tm', 'e2um', 'e2vm', 'e2fm'],
     ('Z',): ['e3tm', 'e3um', 'e3vm', 'e3wm']}

grid = xgcm.Grid(ds, metrics=_metrics, periodic=False) # define xgcm grid
grid_ops = eco.Grid_ops(grid, maskargs={'mask':'nan'}) # define gridoperations. With {'mask':'nan'} operations apply a nan-mask, replacing 0 by nan in data

# define coordinates to pass to diagnostics
_coords = {'X': ds.glamt,
           'Y': ds.gphit, 
           'Z': ds.depth_1d}

properties= eco.Properties(grid_ops, _coords, eos_properties=kwargs_sim['eos'])
energetics= eco.Energetics(grid_ops, properties)
energetics_trend= eco.Energetics_trends(grid_ops,properties,
                                             '../ECOdiagnostics/diagnostics/defaults/processes.ini')

../ECOprocessing/Configs/EXP00_5d50/base.yml
Update processing configuration with sub_5d50


In [4]:
e3t_shift=ds.e3tm.copy().shift(z_c=1).fillna(0)
depth_w =xr.DataArray(data=(- e3t_shift.cumsum('z_c')*ds.mask_bd_t).compute().values, dims=ds.e3wm.dims)

to_ws = grid.interp(ds.to_s,'Z',boundary='extend')
so_ws = grid.interp(ds.so_s,'Z',boundary='extend')
dtdz_w = grid.derivative(ds.to_s,'Z',boundary='extend') 

dh_TZ_w = properties.dh_TZ(to_ws, so_ws, depth_w)
dh_SZ_w = properties.dh_SZ(to_ws, so_ws, depth_w)
dh_TT_w = properties.dh_TT(depth_w,Z_r=-1878)

mask_avt= grid_ops.nan_mask(ds.k_evd)*ds.k_evd/100
mask_avt_invert=(ds.mask_bd_w)*((-(mask_avt-1)).fillna(1))

AttributeError: 'Dataset' object has no attribute 'k_evd'

In [ ]:
#KN2_h_w = (-(dh_TZ_w-dtdz_w*dh_TT_w)*F_tzdf_w-dh_SZ_w*F_szdf_w).compute()
F_tzdf_w=grid.cumint(ds_raw.ttrd_zdf,'Z',boundary='fill',fill_value=0).compute()
F_szdf_w=grid.cumint(ds_raw.strd_zdf,'Z',boundary='fill',fill_value=0).compute()
KN2_h_w_avt =     (-(dh_TZ_w-dtdz_w*dh_TT_w)*F_tzdf_w*mask_avt.fillna(0)- dh_SZ_w*F_szdf_w*mask_avt.fillna(0)).compute()
KN2_h_w_avt_inv = (-(dh_TZ_w-dtdz_w*dh_TT_w)*F_tzdf_w*mask_avt_invert   - dh_SZ_w*F_szdf_w*mask_avt_invert).compute()